# Bmad <-> pyDFCSR 2D 

In [1]:
from pytao import Tao
import yaml

In [2]:
def load_pydfcsr_eles(yamlfile):
    d = yaml.safe_load(open(yamlfile))
    eles = []
    param = {}
    for name, ele in d.items():
        if isinstance(ele, dict):
            eles.append(ele)
            ele['name'] = name
        else:
            param[name] = ele
    return eles

eles = load_pydfcsr_eles('input/chicane_lattice.yaml')

eles

[{'type': 'dipole',
  'L': 0.5002,
  'angle': 0.0483,
  'E1': 0.0,
  'E2': 0.0483,
  'nsep': 1,
  'name': 'B1'},
 {'type': 'drift', 'L': 5.0058, 'nsep': 5, 'name': 'D1'},
 {'type': 'dipole',
  'L': 0.5002,
  'angle': -0.0483,
  'E1': -0.0483,
  'E2': 0.0,
  'nsep': 1,
  'name': 'B2'},
 {'type': 'drift', 'L': 1.0, 'nsep': 1, 'name': 'D2'},
 {'type': 'dipole',
  'L': 0.5002,
  'angle': -0.0483,
  'E1': 0.0,
  'E2': -0.0483,
  'nsep': 1,
  'name': 'B3'},
 {'type': 'drift', 'L': 5.0058, 'nsep': 5, 'name': 'D3'},
 {'type': 'dipole',
  'L': 0.5002,
  'angle': 0.0483,
  'E1': 0.0483,
  'E2': 0,
  'nsep': 1,
  'name': 'B4'},
 {'type': 'drift', 'L': 0.2, 'nsep': 1, 'name': 'Df'}]

# Bmad from PyDFCSR eles 

In [3]:
def bmad_line_from_pydfcsr_ele(ele):
    type = ele['type']
    L = ele['L'] 
    name = ele['name']
    attribs = {}
    if type == 'dipole':
        e1 = ele.get('E1', None)
        key = 'sbend'
        attribs['angle'] = ele['angle']
        attribs['e1'] = ele.get('E1', 0)
        attribs['e2'] = ele.get('E2', 0)
    elif type == 'drift':
        key = 'drift'
    elif type == 'quad':
        key = 'quadrupole'
    else:
        raise NotImplementedError(type)

    
    line = f'{name}: {key}, L = {L}'
    for k, v in attribs.items():
        line = f'{line}, {k}={v}'
    return line

In [4]:
def lat_line(eles, name='lat'):
    names = [ele['name'] for ele in eles]
    return f'{name}: line = (' + (', '.join(names)) + ')'

def bmad_lattice_from__pydfcsr_eles(eles,
                                   e_tot = 10e6,
                                   filename = None
                                   ):
    beles = list(map(bmad_line_from_pydfcsr_ele, eles))

    header = f"""
beginning[beta_a] = 10
beginning[beta_b] = 10
beginning[e_tot] = 100e6
parameter[geometry] = open
parameter[particle] = electron

    """
    latlines = header 
    latlines = latlines + '\n'.join(beles) + '\n'
    latlines = latlines + lat_line(eles)
    latline = latlines + '\nuse, lat'
    return latlines

    if filename:
        with open(filename, 'w') as f:
            for line in latlines:
                f.write(line)
        

In [5]:
bmad_lattice_from__pydfcsr_eles(eles, filename='lat.bmad')

'\nbeginning[beta_a] = 10\nbeginning[beta_b] = 10\nbeginning[e_tot] = 100e6\nparameter[geometry] = open\nparameter[particle] = electron\n\n    B1: sbend, L = 0.5002, angle=0.0483, e1=0.0, e2=0.0483\nD1: drift, L = 5.0058\nB2: sbend, L = 0.5002, angle=-0.0483, e1=-0.0483, e2=0.0\nD2: drift, L = 1.0\nB3: sbend, L = 0.5002, angle=-0.0483, e1=0.0, e2=-0.0483\nD3: drift, L = 5.0058\nB4: sbend, L = 0.5002, angle=0.0483, e1=0.0483, e2=0\nDf: drift, L = 0.2\nlat: line = (B1, D1, B2, D2, B3, D3, B4, Df)'

In [6]:
!cat lat.bmad


beginning[beta_a] = 10
beginning[beta_b] = 10
beginning[e_tot] = 100e6
parameter[geometry] = open
parameter[particle] = electron

B1: sbend, L = 0.5002, angle=0.0483, e1=0.0, e2=0.0483
D1: drift, L = 5.0058
B2: sbend, L = 0.5002, angle=-0.0483, e1=-0.0483, e2=0.0
D2: drift, L = 1.0
B3: sbend, L = 0.5002, angle=-0.0483, e1=0.0, e2=-0.0483
D3: drift, L = 5.0058
B4: sbend, L = 0.5002, angle=0.0483, e1=0.0483, e2=0
Df: drift, L = 0.2
lat: line = (B1, D1, B2, D2, B3, D3, B4, Df)
use, lat

# Run Tao

In [7]:
tao = Tao('-lat lat.bmad -noplot')

In [8]:
def pydfcsr_ele_from_tao_ele(ele):
    
    name = ele['name']
    key = ele['key'].lower()
    L = ele['L'] 

    dfele = {}
    dfele['name'] = name
    dfele['L'] = L
    
    if key == 'sbend':
        type = 'sbend'
        dfele['angle'] = ele['L'] * ele['G']
        dfele['E1'] = ele.get('e1', 0)
        dfele['E2'] = ele.get('e2', 0)
    elif key == 'drift':
        type = 'drift'
    elif key == 'quadrupole':
        type = 'quad'
    else:
        raise NotImplementedError(key)  

    dfele['type'] = type
    return dfele

def pydfcsr_eles_from_tao(tao):

    eles = []
    for name in tao.lat_ele_list():
        bele = tao.ele_head(name)
        bele.update(tao.ele_gen_attribs(name))
        key = bele['key']
        if key.lower() in ('beginning_ele', 'marker' ):
            continue    

        ele = pydfcsr_ele_from_tao_ele(bele)
        eles.append(ele)

    return eles

In [9]:
pydfcsr_eles_from_tao(tao)

[{'name': 'B1',
  'L': 0.5002,
  'angle': 0.04830000000000002,
  'E1': 0,
  'E2': 0,
  'type': 'sbend'},
 {'name': 'D1', 'L': 5.0058, 'type': 'drift'},
 {'name': 'B2',
  'L': 0.5002,
  'angle': -0.04830000000000002,
  'E1': 0,
  'E2': 0,
  'type': 'sbend'},
 {'name': 'D2', 'L': 1.0, 'type': 'drift'},
 {'name': 'B3',
  'L': 0.5002,
  'angle': -0.04830000000000002,
  'E1': 0,
  'E2': 0,
  'type': 'sbend'},
 {'name': 'D3', 'L': 5.0058, 'type': 'drift'},
 {'name': 'B4',
  'L': 0.5002,
  'angle': 0.04830000000000002,
  'E1': 0,
  'E2': 0,
  'type': 'sbend'},
 {'name': 'DF', 'L': 0.2, 'type': 'drift'}]